<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions_attention_and_random_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT DATASETS AND LIBRARIES


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score, f1_score
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import requests
from requests.exceptions import HTTPError
import json as js
from datetime import datetime, timedelta 
import time
from os.path import exists
from decimal import *
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

pd.set_option('display.float_format', lambda x: '%.5f' % x)

#Library

In [4]:
# Function to plot interactive plots using Plotly Express
sc = MinMaxScaler()
num_features = 3
coin_base = False
ku_coin = True
COINBASE_REST_API = 'https://api.pro.coinbase.com' 
COINBASE_PRODUCTS = COINBASE_REST_API+'/products'
KUCOIN_REST_API = "https://api.kucoin.com"
KUCOIN_PRODUCTS = KUCOIN_REST_API+ "/api/v1/market/allTickers"
KUCOIN_CANDLES = KUCOIN_REST_API+ "/api/v1/market/candles"

data_path = '/content/drive/My Drive/output.csv'
model_path = "/content/drive/My Drive/model_ohlc.h5"

def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

def get_single_stock(price_df, vol_df, name):
    return pd.DataFrame({'Date': price_df['Date'], 'Close': price_df[name], 'Volume': vol_df[name]})

def scale_data(data):
  # Scale the data
  scaled_data = sc.fit_transform(data)
  return scaled_data

def sort_date(pric_df):
  pric_df = pric_df.sort_values(by = ['Date'])
  return pric_df

def append_price_dif(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  df = df[:-1]
  return df

def append_price_dif_(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  return df

def append_15d_slope(df):
  df['15Close'] = df['Close'].shift(15)
  df['15Date'] = df['Date'].shift(15)
  df['Trend'] = (df['Close'] - df['15Close']) / 15
  df = df[15:]
  return df

def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

def build_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def connect(url, params):   
  response = requests.get(url,params)
  response.raise_for_status()
  return response

def coinbase_json_to_df(delta, product, granularity='86400'):
  start_date = (datetime.today() - timedelta(seconds=delta*int(granularity))).isoformat()
  end_date = datetime.now().isoformat()
  # Please refer to the coinbase documentation on the expected parameters
  params = {'start':start_date, 'end':end_date, 'granularity':granularity}
  response = connect(COINBASE_PRODUCTS+'/' + product + '/candles', params)
  response_text = response.text
  df_history = pd.read_json(response_text)
  # Add column names in line with the Coinbase Pro documentation
  df_history.columns = ['time','low','high','open','close','volume']
  df_history['time'] = [datetime.fromtimestamp(x) for x in df_history['time']]
  return df_history

def ku_coin_json_to_df(delta, product, granularity='86400'):
  granularity = int(granularity)
  start_date = (datetime.today() - timedelta(seconds=delta*granularity))
  end_date = datetime.now()

  # Please refer to the kucoin documentation on the expected parameters
  params = {'startAt':int(start_date.timestamp()), 'endAt':int(end_date.timestamp()), 'type':gran_to_string(granularity), 'symbol':product}
  response = connect(KUCOIN_CANDLES, params)
  response_text = response.text
  response_data = js.loads(response_text);
  if (response_data["code"] != "200000"):
    raise Exception("Illegal response: " + response_text)
  
  df_history = pd.DataFrame(response_data["data"])

  # kucoin is weird in that they don't have candles for everything. IF we don't have the requested
  # number of bars here, it throws off the whole algo. I don't want to try and project so we 
  # just won't trade those instruments
  got_bars = len(df_history)
  if ( got_bars < delta):
    raise Exception("Requested:" + str(delta) + " bars " + " but only got:" + str(got_bars))

  df_history.columns = ['time','open','close','high','low','volume', 'amount']
  df_history['time'] = [datetime.fromtimestamp(int(x)) for x in df_history['time']]
  df_history['open'] = [float(x) for x in df_history['open']]
  df_history['close'] = [float(x) for x in df_history['close']]
  df_history['high'] = [float(x) for x in df_history['high']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['volume'] = [float(x) for x in df_history['volume']]
  df_history['amount'] = [float(x) for x in df_history['amount']]
  return df_history
  
def gran_to_string(granularity):
  #todo implement this actually
  return "1day"

def get_coin_data_frames(time, product, granularity='86400'):
  if coin_base:
    df_raw = coinbase_json_to_df(time, product, granularity)
  else:
    df_raw = ku_coin_json_to_df(time, product, granularity)

  df_btc_history = df_raw
  if len(df_btc_history.index) == 0:
    print("No data for ", product)
  
  df_btc_history = df_btc_history.rename(columns={"time":"Date", "close":"Close", "volume":"Volume"})
  df_btc_history = sort_date(df_btc_history)
  df_btc_history = df_btc_history.drop(columns={"high", "low", "open"})
  df_btc_history = append_price_dif_(df_btc_history)
  df_btc_history = append_15d_slope(df_btc_history)
  df_btc_features = df_btc_history[["Close", "Volume", "Trend"]]
  df_history_scaled = sc.fit_transform(df_btc_features)
  return [df_btc_history, df_btc_features, df_history_scaled, df_raw]

def build_profit_estimate(predicted, df_btc_history):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_btc_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Predicted-Target"] = df_predicted_chart["Predicted"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted-Target"] - df_predicted_chart["Predicted"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["RealDiff"] = df_btc_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_btc_history["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  profit = df_predicted_chart["Profit"].sum()
  return [df_predicted_chart, profit]

def debug_prediction_frame(predicted, df_history, df_history_scaled):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Original"] = df_history_scaled[:,0]
  df_predicted_chart["Original-Target"] = df_history_scaled[:,2]
  df_predicted_chart["Target-Date"] = df_predicted_chart["Date"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted"] - df_predicted_chart["Original"]
  df_predicted_chart["Actual-Diff"] = df_predicted_chart["Original-Target"] - df_predicted_chart["Original"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["Close"] = df_history["Close"]
  df_predicted_chart["Target"] = df_history["Target"]
  df_predicted_chart["RealDiff"] = df_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_predicted_chart["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  return df_predicted_chart

def get_all_products():
  if coin_base:
    return get_all_coinbase_products()
  
  if ku_coin:
    return get_all_kucoin_products()

def get_all_kucoin_products():
  response = connect(KUCOIN_PRODUCTS, {})
  products = js.loads(response.text)
  df_products = pd.DataFrame(products["data"]["ticker"])
  df_products = df_products.rename(columns={"symbol":"id"})
  return df_products

def get_all_coinbase_products():
  response = connect(COINBASE_PRODUCTS, {})
  response_text = response.text
  df_products = pd.read_json(response_text)
  return df_products

def predict_trade(product, bars):
  [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(bars, product)
  predicted = model.predict(npa_scaled).flatten()
  
  #convert to data frames that have the correct shape for being unscaled
  df_scaled = pd.DataFrame(npa_scaled, columns = ["Close", "Volume", "Trend"])
  
  # I want to believe that scaling happens on a per column basis, we only care about
  # price here so we will dummy out volume and trend and use the scaler on it
  # this kinda sucks, if we add features we'll need to add them here for unscaling
  df_temp = pd.DataFrame(predicted, columns = ["Close"])
  df_temp["Volume"] = 0
  df_temp["Trend"] = 0
  
  # unscale them both
  df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = ["Close", "Volume", "Trend"])
  df_trade = pd.DataFrame(sc.inverse_transform(df_scaled), columns = ["Close", "Volume", "Trend"])
  
  # add predicted
  df_trade["Predicted"] = df_temp["Close"]
  df_trade = df_trade.tail(1)

  # add the product, derive a move and percent
  df_trade["Product"] = row.id;
  df_trade["Move"] = df_trade["Predicted"] - df_trade["Close"]
  df_trade["Percent"] = (df_trade["Move"] / df_trade["Close"]) * 100
  df_trade["RawPercent"] = df_trade["Move"] / df_trade["Close"]
  df_trade["250Fees"] = (250 * 0.004) * 2
  df_trade["5kFees"] = (5000 * 0.004) * 2
  df_trade["10kFees"] = (10000 * 0.0025) * 2
  df_trade["250Profit"] = (250 * df_trade["RawPercent"]) - df_trade["250Fees"] 
  df_trade["5kProfit"] = (5000 * df_trade["RawPercent"]) - df_trade["5kFees"]
  df_trade["10k0Profit"] = (10000 * df_trade["RawPercent"]) - df_trade["10kFees"]
  return df_trade

def get_training_set_for(ticker):
  target_df = get_single_stock(all_stocks_price_df, all_stocks_vol_df, ticker)
  target_df = append_price_dif(target_df)
  target_df = append_15d_slope(target_df)
  features = target_df[["Close", "Volume", "Trend", "Target"]]
  scaled_features = scale_data(features)
  X = []
  y = []
  for i in range(0, len(target_df)):
    X.append(scaled_features [i][0:num_features])
    y.append(scaled_features [i][num_features])
  
  X = np.asarray(X)
  y = np.asarray(y)
  return [scaled_features, X, y]
  
def train_model(model, X, y):

  # One day we might need test, but for now we don't we can use another
  # time series, we have so many
  # Split the data
  #split = int(0.7 * len(X))
  #X_train = X[:split]
  #y_train = y[:split]
  #X_test = X[split:]
  #y_test = y[split:]

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X, (X.shape[0], X.shape[1], 1))
  
  history = model.fit(
      X_train, y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2
  )
  return [model, history]

#pull training data 
all_stocks_price_df = sort_date(pd.read_csv('/content/drive/My Drive/Colab Notebooks/stock.csv'))
all_stocks_vol_df = sort_date(pd.read_csv("/content/drive/My Drive/Colab Notebooks/stock_volume.csv"))

# Get or Train a Model

In [32]:
def create_attention_layer_lstm():

  class AttentionLayer(keras.layers.Layer):
      def __init__(self, **kwargs):
          super(AttentionLayer, self).__init__(**kwargs)

      def build(self, input_shape):
          self.W = self.add_weight(name='att_weight', 
                                  shape=(input_shape[-1], 1),
                                  initializer='normal',
                                  trainable=True)
          self.b = self.add_weight(name='att_bias',
                                  shape=(input_shape[1], 1),
                                  initializer='zeros',
                                  trainable=True)        
          super(AttentionLayer, self).build(input_shape)

      def call(self, inputs):
          e = K.tanh(K.dot(inputs, self.W) + self.b)
          a = K.softmax(e, axis=1)
          output = inputs * a
          return K.sum(output, axis=1)

      def compute_output_shape(self, input_shape):
          return (input_shape[0], input_shape[-1])


  inputs = keras.Input(shape=(3, 1))

  # Define the LSTM layer
  lstm_out = keras.layers.LSTM(32, return_sequences=True)(inputs)

  # Define the attention layer
  attention = AttentionLayer()(lstm_out)

  # Define the output layer
  output = keras.layers.Dense(1, activation='softmax')(attention)

  # Create the model
  model = keras.Model(inputs=inputs, outputs=output)
  return model

In [ ]:
def random_search_model_params():
  [scaled_features, X, y] = get_training_set_for("sp500")  

  def build_variable_length_model(num_layers, num_units):
    features = 3
    outcomes = 1
    model = keras.models.Sequential()
    for i in range(num_layers):
      model.add(keras.layers.LSTM(units=num_units, input_shape=(features, outcomes)))
      model.add(keras.layers.Dense(1))
      model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
    return model

  def random_search_optimal_model():
    # Create the model wrapper
    model = KerasClassifier(build_fn=build_variable_length_model)

    # Define the hyperparameter space
    param_grid = {
    'num_layers': [1, 2, 3],
    'num_units': [32, 64, 128]
    }

    # Create the random search object
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=5)

    # Fit the random search object to the data
    random_search.fit(X, y)

    # Print the best set of hyperparameters
    print(random_search.best_params_)

  random_search_optimal_model()

In [33]:
model = None
file_exists = exists(model_path)

#if file_exists:
#  print("hello")
#  model = keras.models.load_model(model_path)
#else:

[scaled_features, X, y] = get_training_set_for("sp500")  
model_orig = build_model(num_features, 1)
model_searched = create_attention_layer_lstm()
[model_orig, history] = train_model(model_orig, X, y)
[model_searched, history] = train_model(model_orig, X, y)
[btc_history, df_btc_features, df_history_scaled, df_raw] = get_coin_data_frames(180, "BTC-USDT")
loss1 = model_orig.evaluate(df_history_scaled)
loss2 = model_searched.evaluate(df_history_scaled)
print("loss1=", loss1, " loss2=", loss2)


Epoch 1/20
54/54 [==============================] - 8s 52ms/step - loss: 0.0373 - val_loss: 0.0051
Epoch 2/20
54/54 [==============================] - 1s 26ms/step - loss: 0.0022 - val_loss: 0.0071
Epoch 3/20
54/54 [==============================] - 1s 26ms/step - loss: 0.0015 - val_loss: 0.0043
Epoch 4/20
54/54 [==============================] - 1s 26ms/step - loss: 0.0015 - val_loss: 0.0035
Epoch 5/20
54/54 [==============================] - 2s 29ms/step - loss: 0.0013 - val_loss: 0.0029
Epoch 6/20
54/54 [==============================] - 2s 28ms/step - loss: 0.0011 - val_loss: 0.0028
Epoch 7/20
54/54 [==============================] - 1s 27ms/step - loss: 0.0010 - val_loss: 0.0031
Epoch 8/20
54/54 [==============================] - 2s 29ms/step - loss: 9.2137e-04 - val_loss: 0.0033
Epoch 9/20
54/54 [==============================] - 1s 26ms/step - loss: 8.9193e-04 - val_loss: 0.0023
Epoch 10/20
54/54 [==============================] - 1s 28ms/step - loss: 7.6346e-04 - val_loss: 0.00

In [5]:
# additional training?
if 0:
  [scaled_features, X, y] = get_training_set_for("IBM")  
  [model, history] = train_model(model, X, y)


# Visualize and Backtest

In [34]:
budget = 5000
[btc_history, df_btc_features, df_history_scaled, df_raw] = get_coin_data_frames(180, "FCON-USDT")
predicted = model_searched.predict(df_history_scaled).flatten()
predicted2 = model_orig.predict(df_history_scaled).flatten()
[df_profit, profit] = build_profit_estimate(predicted, btc_history)
[df_profit, profit2] = build_profit_estimate(predicted2, btc_history)
df_chart = debug_prediction_frame(predicted, btc_history, df_history_scaled)
df_chart["Predicted-Orig"] = predicted2
interactive_plot(df_chart[["Date","Original", "Predicted-Orig"]], "Wtf")
interactive_plot(df_chart[["Date","Original", "Predicted"]], "Wtf")
print("Profit searched:", profit)
print("Profit orig:", profit2)
df_chart


6/6 [==============================] - 0s 9ms/step


Profit searched: 13063.42369568083
Profit orig: 13063.42369568083


,Date,Predicted,Original,Original-Target,Target-Date,Predicted-Diff,Actual-Diff,Should-Trade,Close,Target,RealDiff,Percent,Profit,Predicted-Orig
164,2022-07-18,0.84783,0.86496,0.37058,2022-07-19,-0.01712,-0.49438,False,0.00068,0.00068,0.00000,0.00000,0.00000,0.84783
163,2022-07-19,0.84748,0.86496,0.38614,2022-07-20,-0.01748,-0.47882,False,0.00068,0.00065,-0.00003,-0.03840,0.00000,0.84748
162,2022-07-20,0.82052,0.82949,0.33663,2022-07-21,-0.00898,-0.49286,False,0.00065,0.00064,-0.00001,-0.01843,0.00000,0.82052
161,2022-07-21,0.80706,0.81312,0.33380,2022-07-22,-0.00606,-0.47932,False,0.00064,0.00064,0.00001,0.00782,0.00000,0.80706
160,2022-07-22,0.81262,0.81994,0.34088,2022-07-23,-0.00732,-0.47907,False,0.00064,0.00063,-0.00002,-0.02329,0.00000,0.81262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-12-25,-0.02428,0.00259,0.34767,2022-12-26,-0.02687,0.34507,False,0.00004,0.00005,0.00000,0.04688,0.00000,-0.02428
3,2022-12-26,-0.02127,0.00546,0.33649,2022-12-27,-0.02672,0.33104,False,0.00005,0.00004,-0.00000,-0.04264,0.00000,-0.02127
2,2022-12-27,-0.02591,0.00273,0.32942,2022-12-28,-0.02864,0.32669,False,0.00004,0.00004,-0.00000,-0.04454,0.00000,-0.02591
1,2022-12-28,-0.02972,0.00000,0.34272,2022-12-29,-0.02972,0.34272,False,0.00004,0.00005,0.00000,0.10723,0.00000,-0.02972


In [7]:
print("rscore: ", r2_score(predicted, df_history_scaled[:, 2]))



rscore:  0.13113149877319585


# Which coins are most profitable based on the above trading signals?

In [8]:
# download all known products and check who has the highest profit in 90 days
"""
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USD')]
df_products = df_products[df_products.trading_disabled == False]
df_products = df_products[df_products.cancel_only == False]
df_profit = pd.DataFrame();
df_profit["Product"] = [];
df_profit["Profit"] = [];
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)
    predicted = model.predict(df_scaled.flatten())
    [df_chart, profit] = build_profit_estimate(predicted, df_full)
    df_profit.loc[len(df_profit.index)] = [row.id, profit] 
  except Exception as inst:
    print("Error: ", inst)
  time.sleep(1)

df_profit
"""

'\n# Fetch the top 10 and see if they predict up\ndf_products = get_all_products()\ndf_products = df_products[df_products.id.str.endswith(\'USD\')]\ndf_products = df_products[df_products.trading_disabled == False]\ndf_products = df_products[df_products.cancel_only == False]\ndf_profit = pd.DataFrame();\ndf_profit["Product"] = [];\ndf_profit["Profit"] = [];\nfor index, row in df_products.iterrows():\n  try:\n    print("fetching: ", row.id)\n    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)\n    predicted = model.predict(df_scaled.flatten())\n    [df_chart, profit] = build_profit_estimate(predicted, df_full)\n    df_profit.loc[len(df_profit.index)] = [row.id, profit] \n  except Exception as inst:\n    print("Error: ", inst)\n  time.sleep(1)\n\ndf_profit\n'

# What has a buy indicator for tomorrow?

In [9]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
bars = 90
counter = 0;
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    
    df_trade = predict_trade(row.id, bars)
   
    # we need to unscale the predicted values so that we have an entry and exit point
    # entry should be roughly close and exit should be roughly predicted

    # Stick this on the end of the main dataframe
    df_trades = df_trades.append(df_trade);
    
    #counter+=1
    #if counter > 5:
    #  break
  except Exception as inst:
    #raise inst
    print("Error: ", inst)
  time.sleep(0.5)
df_trades.reset_index()
df_buys = df_trades[df_trades['Move'] > 0] 
df_shorts = df_trades[df_trades['Move'] < 0] 




fetching:  NKN-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  GEM-USDT
Error:  Requested:90 bars  but only got:80
fetching:  CUSD-USDT
3/3 [==============================] - 0s 14ms/step
fetching:  LTC3L-USDT
3/3 [==============================] - 0s 12ms/step
fetching:  OAS-USDT
Error:  Requested:90 bars  but only got:18
fetching:  KNC-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  LYM-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  HAI-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  MITX-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  PDEX-USDT
Error:  429 Client Error: Too Many Requests for url: https://api.kucoin.com/api/v1/market/candles?startAt=1664536945&endAt=1672312945&type=1day&symbol=PDEX-USDT
fetching:  FLAME-USDT
3/3 [==============================] - 0s 12ms/step
fetching:  EPX-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  AOG-USDT
3/3 [=============

In [13]:
df_buys

,Close,Volume,Trend,Predicted,Product,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
74,0.49700,37668.52830,-0.00833,0.49926,KNC-USDT,0.00226,0.45569,0.00456,2.00000,40.00000,50.00000,-0.86078,-17.21551,-4.43101
74,0.00299,7677510.27870,-0.00003,0.00299,MITX-USDT,0.00000,0.09413,0.00094,2.00000,40.00000,50.00000,-1.76467,-35.29341,-40.58683
74,0.02123,4833296.03470,-0.00010,0.02135,FLAME-USDT,0.00012,0.56480,0.00565,2.00000,40.00000,50.00000,-0.58801,-11.76024,6.47953
74,0.07389,3067.35990,-0.00025,0.07394,YLD-USDT,0.00005,0.07379,0.00074,2.00000,40.00000,50.00000,-1.81551,-36.31029,-42.62057
74,0.21500,88328.44870,-0.00347,0.21601,CWS-USDT,0.00101,0.46893,0.00469,2.00000,40.00000,50.00000,-0.82768,-16.55368,-3.10735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.00064,51289677.06530,-0.00002,0.00064,LITH-USDT,0.00000,0.44233,0.00442,2.00000,40.00000,50.00000,-0.89418,-17.88366,-5.76732
74,0.00144,21456305.58800,-0.00003,0.00146,2CRZ-USDT,0.00002,1.46680,0.01467,2.00000,40.00000,50.00000,1.66701,33.34016,96.68031
74,0.43050,292097.89010,-0.00486,0.43101,RNDR-USDT,0.00051,0.11886,0.00119,2.00000,40.00000,50.00000,-1.70286,-34.05723,-38.11445
74,0.00277,47953811.11690,-0.00005,0.00279,DMTR-USDT,0.00002,0.88802,0.00888,2.00000,40.00000,50.00000,0.22006,4.40112,38.80225


In [11]:
df_trade = predict_trade("REN-USD", 90)
df_trade

Exception: ignored

In [ ]:
with open(data_path, 'w', encoding = 'utf-8-sig') as f:
  df_trades.to_csv(f)

In [ ]:
model.save(model_path)

In [ ]:
df_shorts[df_shorts["Product"] == "MPL-USD"]